In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
img_dir = ('/content/drive/MyDrive/image_classifier/dataset')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import matplotlib.pyplot as plt

In [ ]:
data_args = dict(rescale=1./255, validation_split=.20)

In [ ]:
bag_train = tf.keras.preprocessing.image.ImageDataGenerator(**data_args)

train_gen = bag_train.flow_from_directory(
img_dir,
subset="training",
shuffle=True,
target_size=(224, 224))


bag_val = tf.keras.preprocessing.image.ImageDataGenerator(**data_args)

val_gen = bag_val.flow_from_directory(
img_dir,
subset="validation",
shuffle=True,
target_size=(224, 224))

In [ ]:
for image_batch, label_batch in train_gen:
  break
  image_batch.shape, label_batch.shape

In [ ]:
print (train_gen.class_indices)

In [ ]:
base_model = tf.keras.Sequential([
 hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
 output_shape=[1280],
 trainable=False),
 tf.keras.layers.Dropout(0.4),
 tf.keras.layers.Dense(train_gen.num_classes, activation='softmax')
])
base_model.build([None, 224, 224, 3])
base_model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)

base_model.compile(
 optimizer=optimizer,
 loss= 'categorical_crossentropy',
 metrics=['accuracy'])

In [ ]:
epochs=10
history = base_model.fit(
  train_gen,
  validation_data = val_gen,
  epochs = epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import pickle
from sklearn.externals import joblib 

In [ ]:
file_dir = '/content/drive/MyDrive/image_classifier/model.h5'
base_model.save(file_dir)